In [ ]:
# !pip install torch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 --index-url https://download.pytorch.org/whl/cpu

https://parsio.io/blog/extracting-data-from-pdfs-using-ai-claude-3-donut-and-nougat/

### Convert PDF to Images

In [4]:
from pdf2image import convert_from_path

# Store Pdf with convert_from_path function
images = convert_from_path('./data.pdf', poppler_path = r"C:\Program Files\poppler-24.02.0\Library\bin")

for i in range(len(images)):
	# Save pages as images in the pdf
	images[i].save('./images/page_'+ str(i) +'.jpg', 'JPEG')

### OCR Tables in each image Using Donut

In [1]:
import re
from transformers import DonutProcessor, VisionEncoderDecoderModel
import json
import torch
from PIL import Image

c:\Users\hp\Documents\Github\document-parser-donut\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Load the processor and model
processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base-finetuned-cord-v2")
MODEL = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base-finetuned-cord-v2")

# Move model to the appropriate device
device = "cuda" if torch.cuda.is_available() else "cpu"
MODEL.to(device)

c:\Users\hp\Documents\Github\document-parser-donut\venv\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hp\.cache\huggingface\hub\models--naver-clova-ix--donut-base-finetuned-cord-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


VisionEncoderDecoderModel(
  (encoder): DonutSwinModel(
    (embeddings): DonutSwinEmbeddings(
      (patch_embeddings): DonutSwinPatchEmbeddings(
        (projection): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      )
      (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): DonutSwinEncoder(
      (layers): ModuleList(
        (0): DonutSwinStage(
          (blocks): ModuleList(
            (0-1): 2 x DonutSwinLayer(
              (layernorm_before): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
              (attention): DonutSwinAttention(
                (self): DonutSwinSelfAttention(
                  (query): Linear(in_features=128, out_features=128, bias=True)
                  (key): Linear(in_features=128, out_features=128, bias=True)
                  (value): Linear(in_features=128, out_features=128, bias=True)
                  (dropout): Dropout(p=0.0, inplace=False)
           

In [8]:
IMG_PATH = "./images/page_1.jpg"

In [12]:
def process_image(img_path, model):
    image = Image.open(img_path)

    # prepare decoder inputs
    task_prompt = "<s_cord-v2>"
    decoder_input_ids = processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt").input_ids

    pixel_values = processor(image, return_tensors="pt").pixel_values

    outputs = model.generate(
        pixel_values.to(device),
        decoder_input_ids=decoder_input_ids.to(device),
        max_length=model.decoder.config.max_position_embeddings,
        pad_token_id=processor.tokenizer.pad_token_id,
        eos_token_id=processor.tokenizer.eos_token_id,
        use_cache=True,
        bad_words_ids=[[processor.tokenizer.unk_token_id]],
        return_dict_in_generate=True,
    )

    sequence = processor.batch_decode(outputs.sequences)[0]
    sequence = sequence.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "")
    sequence = re.sub(r"<.*?>", "", sequence, count=1).strip()  # remove first task start token

    parsed_dict = processor.token2json(sequence)
    # print the prettified dictionary
    prettified_dict = json.dumps(parsed_dict, indent=4)
    print(prettified_dict)

    return prettified_dict

In [13]:
prettified_dict = process_image(IMG_PATH, MODEL)

[
    {
        "nm": "SAUZE",
        "num": "DECESSION SUSHIZE"
    },
    {
        "nm": "SAUZE"
    }
]


In [17]:
!pip freeze > requirements.txt